In [2]:
import tensorflow_federated as tff
import tensorflow as tf
import numpy as np
from scipy.stats import ks_2samp, chi2_contingency
import nest_asyncio

nest_asyncio.apply()

# Set the local execution context
tff.backends.native.set_local_execution_context()

# Load and preprocess the MNIST dataset
def preprocess(dataset):
    return dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)))

mnist_train, mnist_test = tf.keras.datasets.mnist.load_data()
mnist_train = (mnist_train[0].reshape(-1, 28, 28), mnist_train[1])
mnist_test = (mnist_test[0].reshape(-1, 28, 28), mnist_test[1])

# Split the data into 10 clients
def create_client_data(data, labels, num_clients=10):
    client_data = []
    client_labels = []
    data_per_client = len(data) // num_clients
    for i in range(num_clients):
        client_data.append(data[i * data_per_client:(i + 1) * data_per_client])
        client_labels.append(labels[i * data_per_client:(i + 1) * data_per_client])
    return client_data, client_labels

client_data, client_labels = create_client_data(mnist_train[0], mnist_train[1])

# Define the metrics function
def perform_differential_testing(predictions_i, predictions_j):
    if predictions_i.ndim == 1:
        predictions_i = np.expand_dims(predictions_i, axis=1)
    if predictions_j.ndim == 1:
        predictions_j = np.expand_dims(predictions_j, axis=1)
    
    pred_class_i = np.argmax(predictions_i, axis=1)
    pred_class_j = np.argmax(predictions_j, axis=1)
    
    Δ_class = np.sum(pred_class_i != pred_class_j)
    Δ_score = np.sum(predictions_i != predictions_j)
    P_KS = ks_2samp(predictions_i.flatten(), predictions_j.flatten()).pvalue
    contingency = np.array([[np.sum((pred_class_i == k) & (pred_class_j == l)) for l in range(10)] for k in range(10)])
    contingency += 1  # Add-one smoothing
    P_X2 = chi2_contingency(contingency)[1]

    return Δ_class, Δ_score, P_KS, P_X2

# Create a simple model
def create_model():
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# Create a federated learning process
def model_fn():
    model = create_model()
    return tff.learning.from_keras_model(
        model,
        input_spec=(tf.TensorSpec(shape=[None, 28, 28], dtype=tf.float32),
                    tf.TensorSpec(shape=[None], dtype=tf.int32)),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Define the client optimizer function
def client_optimizer_fn():
    return tf.keras.optimizers.Nadam(learning_rate=0.001)

# Define the federated averaging process
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=client_optimizer_fn
)

# Initialize the process
state = iterative_process.initialize()

# Custom function to determine if a model is an outlier using a DBSCAN-like approach
def is_outlier(metric_data, epsilon=0.4, min_samples=3):
    num_points = metric_data.shape[0]
    distances = np.linalg.norm(metric_data[:, np.newaxis] - metric_data, axis=2)
    neighbors = np.sum(distances < epsilon, axis=1)
    outliers = neighbors < min_samples
    return outliers

# Standalone function for preprocessing
def preprocess_fn(x, y):
    return tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)

# Simulate federated training
num_rounds = 10 # Define the number of rounds
num_clients = 10  # Define the number of clients

for round_num in range(1, num_rounds + 1):
    # Create TensorFlow datasets for each client
    federated_data = [
        tf.data.Dataset.from_tensor_slices((client_data[i], client_labels[i]))
        .map(preprocess_fn)
        .batch(20)
        for i in range(num_clients)
    ]
    
    # Perform a round of federated training
    state, metrics = iterative_process.next(state, federated_data)
    print(f'Round {round_num}, Metrics: {metrics}')
    
    # Get predictions for each client
    predictions = []
    for i in range(num_clients):
        model = create_model()
        learning_rate = 10.0 if i == 0 else 0.001  # Introduce a bug in the first client
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        model.fit(client_data[i], client_labels[i], epochs=1, verbose=0)
        predictions.append(model.predict(client_data[i]))
    
    # Calculate the delta class matrix
    delta_class_matrix = np.zeros((num_clients, num_clients))
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            Δ_class, _, _, _ = perform_differential_testing(predictions[i], predictions[j])
            delta_class_matrix[i, j] = Δ_class
            delta_class_matrix[j, i] = Δ_class  # Mirror the matrix
    
    # Print the delta class matrix
    print(f"Round {round_num} Delta Class Matrix:")
    print(delta_class_matrix)
    
    # Calculate the distance matrix
    distance_matrix = np.linalg.norm(delta_class_matrix[:, np.newaxis] - delta_class_matrix, axis=2)
    
    # Print the distance matrix
    print(f"Round {round_num} Distance Matrix:")
    print(distance_matrix)
    
    # Detect outliers using the custom DBSCAN-like function
    outliers = is_outlier(distance_matrix)
    print(f'Round {round_num}, Outliers: {outliers}')

Round 1, Metrics: OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.83365), ('loss', 0.60873526)]))])
Round 1 Delta Class Matrix:
[[   0. 5456. 5539. 5371. 5580. 5345. 5483. 5510. 5438. 5451.]
 [5456.    0. 5407. 5365. 5409. 5399. 5386. 5401. 5399. 5387.]
 [5539. 5407.    0. 5377. 5374. 5407. 5370. 5408. 5351. 5385.]
 [5371. 5365. 5377.    0. 5425. 5437. 5400. 5415. 5411. 5409.]
 [5580. 5409. 5374. 5425.    0. 5374. 5404. 5418. 5358. 5389.]
 [5345. 5399. 5407. 5437. 5374.    0. 5381. 5399. 5373. 5379.]
 [5483. 5386. 5370. 5400. 5404. 5381.    0. 5450. 5392. 5348.]
 [5510. 5401. 5408. 5415. 5418. 5399. 5450.    0. 5401. 5432.]
 [5438. 5399. 5351. 5411. 5358. 5373. 5392. 5401.    0. 5414.]
 [5451. 5387. 5385. 5409. 5389. 5379. 5348. 5432. 5414.    0.]]
Round 1 Distance Matrix:
[[   0.         7720.90642347 7838.67699296 7601.36191745 7894.99461684
  7565.55675413 

new approach 

delta class

In [8]:
import tensorflow_federated as tff
import tensorflow as tf
import numpy as np
from scipy.stats import ks_2samp, chi2_contingency
import nest_asyncio

nest_asyncio.apply()

# Set the local execution context
tff.backends.native.set_local_execution_context()

# Load and preprocess the MNIST dataset
def preprocess(dataset):
    return dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)))

mnist_train, mnist_test = tf.keras.datasets.mnist.load_data()
mnist_train = (mnist_train[0].reshape(-1, 28, 28), mnist_train[1])
mnist_test = (mnist_test[0].reshape(-1, 28, 28), mnist_test[1])

# Split the data into 10 clients
def create_client_data(data, labels, num_clients=10):
    client_data = []
    client_labels = []
    data_per_client = len(data) // num_clients
    for i in range(num_clients):
        client_data.append(data[i * data_per_client:(i + 1) * data_per_client])
        client_labels.append(labels[i * data_per_client:(i + 1) * data_per_client])
    return client_data, client_labels

client_data, client_labels = create_client_data(mnist_train[0], mnist_train[1])

# Define the metrics function
def perform_differential_testing(predictions_i, predictions_j):
    if predictions_i.ndim == 1:
        predictions_i = np.expand_dims(predictions_i, axis=1)
    if predictions_j.ndim == 1:
        predictions_j = np.expand_dims(predictions_j, axis=1)
    
    pred_class_i = np.argmax(predictions_i, axis=1)
    pred_class_j = np.argmax(predictions_j, axis=1)
    
    Δ_class = np.sum(pred_class_i != pred_class_j)
    Δ_score = np.sum(predictions_i != predictions_j)
    P_KS = ks_2samp(predictions_i.flatten(), predictions_j.flatten()).pvalue
    contingency = np.array([[np.sum((pred_class_i == k) & (pred_class_j == l)) for l in range(10)] for k in range(10)])
    contingency += 1  # Add-one smoothing
    P_X2 = chi2_contingency(contingency)[1]

    return Δ_class, Δ_score, P_KS, P_X2

# Create a simple model
def create_model():
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# Create a federated learning process
def model_fn():
    model = create_model()
    return tff.learning.from_keras_model(
        model,
        input_spec=(tf.TensorSpec(shape=[None, 28, 28], dtype=tf.float32),
                    tf.TensorSpec(shape=[None], dtype=tf.int32)),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Define the client optimizer function
def client_optimizer_fn():
    return tf.keras.optimizers.Nadam(learning_rate=0.001)

# Define the federated averaging process
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=client_optimizer_fn
)

# Initialize the process
state = iterative_process.initialize()

# Custom function to determine if a model is an outlier using a DBSCAN-like approach
def is_outlier(metric_data, epsilon=0.5, min_samples=2):
    num_points = metric_data.shape[0]
    distances = np.linalg.norm(metric_data[:, np.newaxis] - metric_data, axis=2)
    neighbors = np.sum(distances < epsilon, axis=1)
    outliers = neighbors < min_samples
    return outliers

# Standalone function for preprocessing
def preprocess_fn(x, y):
    return tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)

# Simulate federated training
num_rounds = 10  # Define the number of rounds
num_clients = 10  # Define the number of clients

for round_num in range(1, num_rounds + 1):
    # Create TensorFlow datasets for each client
    federated_data = [
        tf.data.Dataset.from_tensor_slices((client_data[i], client_labels[i]))
        .map(preprocess_fn)
        .batch(20)
        for i in range(num_clients)
    ]
    
    # Perform a round of federated training
    state, metrics = iterative_process.next(state, federated_data)
    print(f'Round {round_num}, Metrics: {metrics}')
    
    # Get predictions for each client
    predictions = []
    for i in range(num_clients):
        model = create_model()
        learning_rate = 100.0 if i == 0 else 0.001  # Introduce a bug in the first client
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        model.fit(client_data[i], client_labels[i], epochs=1, verbose=0)
        predictions.append(model.predict(client_data[i]))
    
    # Calculate the delta class matrix
    delta_class_matrix = np.zeros((num_clients, num_clients))
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            Δ_class, _, _, _ = perform_differential_testing(predictions[i], predictions[j])
            delta_class_matrix[i, j] = Δ_class
            delta_class_matrix[j, i] = Δ_class  # Mirror the matrix
    
    # Print the delta class matrix
    print(f"Round {round_num} Delta Class Matrix:")
    print(delta_class_matrix)
    
    # Calculate the distance matrix
    distance_matrix = np.linalg.norm(delta_class_matrix[:, np.newaxis] - delta_class_matrix, axis=2)
    
    # Print the distance matrix
    print(f"Round {round_num} Distance Matrix:")
    print(distance_matrix)
    
    # Calculate and print average distances
    average_distances = np.mean(distance_matrix, axis=1)
    print(f"Round {round_num} Average distances:")
    print(average_distances)
    
    # Detect outliers using the custom DBSCAN-like function
    outliers = is_outlier(distance_matrix)
    print(f'Round {round_num}, Outliers: {outliers}')
    
    # Identify the client with the highest average distance
    max_distance_client = np.argmax(average_distances)
    print(f"Round {round_num}, Client with highest average distance: {max_distance_client}")

Round 1, Metrics: OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.83633333), ('loss', 0.5986855)]))])
Round 1 Delta Class Matrix:
[[   0. 5390. 5443. 5389. 5418. 5496. 5408. 5449. 5530. 5425.]
 [5390.    0. 5426. 5386. 5424. 5401. 5379. 5404. 5405. 5406.]
 [5443. 5426.    0. 5395. 5392. 5416. 5367. 5385. 5359. 5361.]
 [5389. 5386. 5395.    0. 5418. 5428. 5410. 5417. 5419. 5425.]
 [5418. 5424. 5392. 5418.    0. 5388. 5385. 5385. 5400. 5404.]
 [5496. 5401. 5416. 5428. 5388.    0. 5408. 5408. 5374. 5357.]
 [5408. 5379. 5367. 5410. 5385. 5408.    0. 5406. 5365. 5362.]
 [5449. 5404. 5385. 5417. 5385. 5408. 5406.    0. 5408. 5420.]
 [5530. 5405. 5359. 5419. 5400. 5374. 5365. 5408.    0. 5375.]
 [5425. 5406. 5361. 5425. 5404. 5357. 5362. 5420. 5375.    0.]]
Round 1 Distance Matrix:
[[   0.         7624.46135802 7700.65062186 7622.52812392 7664.70325583
  7774.6989652

delta score


In [9]:
import tensorflow_federated as tff
import tensorflow as tf
import numpy as np
from scipy.stats import ks_2samp, chi2_contingency
import nest_asyncio

nest_asyncio.apply()

# Set the local execution context
tff.backends.native.set_local_execution_context()

# Load and preprocess the MNIST dataset
def preprocess(dataset):
    return dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)))

mnist_train, mnist_test = tf.keras.datasets.mnist.load_data()
mnist_train = (mnist_train[0].reshape(-1, 28, 28), mnist_train[1])
mnist_test = (mnist_test[0].reshape(-1, 28, 28), mnist_test[1])

# Split the data into 10 clients
def create_client_data(data, labels, num_clients=10):
    client_data = []
    client_labels = []
    data_per_client = len(data) // num_clients
    for i in range(num_clients):
        client_data.append(data[i * data_per_client:(i + 1) * data_per_client])
        client_labels.append(labels[i * data_per_client:(i + 1) * data_per_client])
    return client_data, client_labels

client_data, client_labels = create_client_data(mnist_train[0], mnist_train[1])

# Define the metrics function
def perform_differential_testing(predictions_i, predictions_j):
    if predictions_i.ndim == 1:
        predictions_i = np.expand_dims(predictions_i, axis=1)
    if predictions_j.ndim == 1:
        predictions_j = np.expand_dims(predictions_j, axis=1)
    
    Δ_class = np.sum(np.argmax(predictions_i, axis=1) != np.argmax(predictions_j, axis=1))
    Δ_score = np.sum(predictions_i != predictions_j)
    P_KS = ks_2samp(predictions_i.flatten(), predictions_j.flatten()).pvalue
    contingency = np.array([[np.sum((np.argmax(predictions_i, axis=1) == k) & (np.argmax(predictions_j, axis=1) == l)) for l in range(10)] for k in range(10)])
    contingency += 1  # Add-one smoothing
    P_X2 = chi2_contingency(contingency)[1]

    return Δ_class, Δ_score, P_KS, P_X2

# Create a simple model
def create_model():
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# Create a federated learning process
def model_fn():
    model = create_model()
    return tff.learning.from_keras_model(
        model,
        input_spec=(tf.TensorSpec(shape=[None, 28, 28], dtype=tf.float32),
                    tf.TensorSpec(shape=[None], dtype=tf.int32)),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Define the client optimizer function
def client_optimizer_fn():
    return tf.keras.optimizers.Nadam(learning_rate=0.001)

# Define the federated averaging process
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=client_optimizer_fn
)

# Initialize the process
state = iterative_process.initialize()

# Custom function to determine if a model is an outlier using a DBSCAN-like approach
def is_outlier(metric_data, epsilon=0.5, min_samples=2):
    num_points = metric_data.shape[0]
    distances = np.linalg.norm(metric_data[:, np.newaxis] - metric_data, axis=2)
    neighbors = np.sum(distances < epsilon, axis=1)
    outliers = neighbors < min_samples
    return outliers

# Standalone function for preprocessing
def preprocess_fn(x, y):
    return tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)

# Simulate federated training
num_rounds = 10  # Define the number of rounds
num_clients = 10  # Define the number of clients

for round_num in range(1, num_rounds + 1):
    # Create TensorFlow datasets for each client
    federated_data = [
        tf.data.Dataset.from_tensor_slices((client_data[i], client_labels[i]))
        .map(preprocess_fn)
        .batch(20)
        for i in range(num_clients)
    ]
    
    # Perform a round of federated training
    state, metrics = iterative_process.next(state, federated_data)
    print(f'Round {round_num}, Metrics: {metrics}')
    
    # Get predictions for each client
    predictions = []
    for i in range(num_clients):
        model = create_model()
        learning_rate = 10.0 if i == 0 else 0.001  # Introduce a bug in the first client
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        model.fit(client_data[i], client_labels[i], epochs=1, verbose=0)
        predictions.append(model.predict(client_data[i]))
    
    # Calculate the delta score matrix
    delta_score_matrix = np.zeros((num_clients, num_clients))
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            _, Δ_score, _, _ = perform_differential_testing(predictions[i], predictions[j])
            delta_score_matrix[i, j] = Δ_score
            delta_score_matrix[j, i] = Δ_score  # Mirror the matrix
    
    # Print the delta score matrix
    print(f"Round {round_num} Delta Score Matrix:")
    print(delta_score_matrix)
    
    # Calculate the distance matrix
    distance_matrix = np.linalg.norm(delta_score_matrix[:, np.newaxis] - delta_score_matrix, axis=2)
    
    # Print the distance matrix
    print(f"Round {round_num} Distance Matrix:")
    print(distance_matrix)
    
    # Calculate and print average distances
    average_distances = np.mean(distance_matrix, axis=1)
    print(f"Round {round_num} Average distances:")
    print(average_distances)
    
    # Detect outliers using the custom DBSCAN-like function
    outliers = is_outlier(distance_matrix)
    print(f'Round {round_num}, Outliers: {outliers}')
    
    # Identify the client with the highest average distance
    max_distance_client = np.argmax(average_distances)
    print(f"Round {round_num}, Client with highest average distance: {max_distance_client}")

Round 1, Metrics: OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.84076667), ('loss', 0.5923675)]))])
Round 1 Delta Score Matrix:
[[    0. 59990. 59978. 59977. 59977. 59984. 59981. 59982. 59985. 59977.]
 [59990.     0. 38623. 38034. 38903. 39696. 41697. 39439. 39407. 36399.]
 [59978. 38623.     0. 30113. 31477. 32720. 35414. 32171. 32158. 28004.]
 [59977. 38034. 30113.     0. 30760. 31847. 34686. 31269. 31721. 27051.]
 [59977. 38903. 31477. 30760.     0. 33117. 35781. 32438. 32719. 28433.]
 [59984. 39696. 32720. 31847. 33117.     0. 36769. 33590. 33872. 29775.]
 [59981. 41697. 35414. 34686. 35781. 36769.     0. 36322. 36291. 32725.]
 [59982. 39439. 32171. 31269. 32438. 33590. 36322.     0. 33468. 29151.]
 [59985. 39407. 32158. 31721. 32719. 33872. 36291. 33468.     0. 29525.]
 [59977. 36399. 28004. 27051. 28433. 29775. 32725. 29151. 29525.     0.]]
Round 1 Dis

PKS 

In [10]:
import tensorflow_federated as tff
import tensorflow as tf
import numpy as np
from scipy.stats import ks_2samp, chi2_contingency
import nest_asyncio

nest_asyncio.apply()

# Set the local execution context
tff.backends.native.set_local_execution_context()

# Load and preprocess the MNIST dataset
def preprocess(dataset):
    return dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)))

mnist_train, mnist_test = tf.keras.datasets.mnist.load_data()
mnist_train = (mnist_train[0].reshape(-1, 28, 28), mnist_train[1])
mnist_test = (mnist_test[0].reshape(-1, 28, 28), mnist_test[1])

# Split the data into 10 clients
def create_client_data(data, labels, num_clients=10):
    client_data = []
    client_labels = []
    data_per_client = len(data) // num_clients
    for i in range(num_clients):
        client_data.append(data[i * data_per_client:(i + 1) * data_per_client])
        client_labels.append(labels[i * data_per_client:(i + 1) * data_per_client])
    return client_data, client_labels

client_data, client_labels = create_client_data(mnist_train[0], mnist_train[1])

# Define the metrics function
def perform_differential_testing(predictions_i, predictions_j):
    if predictions_i.ndim == 1:
        predictions_i = np.expand_dims(predictions_i, axis=1)
    if predictions_j.ndim == 1:
        predictions_j = np.expand_dims(predictions_j, axis=1)
    
    Δ_class = np.sum(np.argmax(predictions_i, axis=1) != np.argmax(predictions_j, axis=1))
    Δ_score = np.sum(predictions_i != predictions_j)
    P_KS = ks_2samp(predictions_i.flatten(), predictions_j.flatten()).pvalue
    contingency = np.array([[np.sum((np.argmax(predictions_i, axis=1) == k) & (np.argmax(predictions_j, axis=1) == l)) for l in range(10)] for k in range(10)])
    contingency += 1  # Add-one smoothing
    P_X2 = chi2_contingency(contingency)[1]

    return Δ_class, Δ_score, P_KS, P_X2

# Create a simple model
def create_model():
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# Create a federated learning process
def model_fn():
    model = create_model()
    return tff.learning.from_keras_model(
        model,
        input_spec=(tf.TensorSpec(shape=[None, 28, 28], dtype=tf.float32),
                    tf.TensorSpec(shape=[None], dtype=tf.int32)),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Define the client optimizer function
def client_optimizer_fn():
    return tf.keras.optimizers.Nadam(learning_rate=0.001)

# Define the federated averaging process
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=client_optimizer_fn
)

# Initialize the process
state = iterative_process.initialize()

# Custom function to determine if a model is an outlier using a DBSCAN-like approach
def is_outlier(metric_data, epsilon=0.5, min_samples=2):
    num_points = metric_data.shape[0]
    distances = np.linalg.norm(metric_data[:, np.newaxis] - metric_data, axis=2)
    neighbors = np.sum(distances < epsilon, axis=1)
    outliers = neighbors < min_samples
    return outliers

# Standalone function for preprocessing
def preprocess_fn(x, y):
    return tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)

# Simulate federated training
num_rounds = 10  # Define the number of rounds
num_clients = 10  # Define the number of clients

for round_num in range(1, num_rounds + 1):
    # Create TensorFlow datasets for each client
    federated_data = [
        tf.data.Dataset.from_tensor_slices((client_data[i], client_labels[i]))
        .map(preprocess_fn)
        .batch(20)
        for i in range(num_clients)
    ]
    
    # Perform a round of federated training
    state, metrics = iterative_process.next(state, federated_data)
    print(f'Round {round_num}, Metrics: {metrics}')
    
    # Get predictions for each client
    predictions = []
    for i in range(num_clients):
        model = create_model()
        learning_rate = 10.0 if i == 0 else 0.001  # Introduce a bug in the first client
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        model.fit(client_data[i], client_labels[i], epochs=1, verbose=0)
        predictions.append(model.predict(client_data[i]))
    
    # Calculate the P_KS matrix
    p_ks_matrix = np.zeros((num_clients, num_clients))
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            _, _, p_ks, _ = perform_differential_testing(predictions[i], predictions[j])
            p_ks_matrix[i, j] = p_ks
            p_ks_matrix[j, i] = p_ks  # Mirror the matrix
    
    # Print the P_KS matrix
    print(f"Round {round_num} P_KS Matrix:")
    print(p_ks_matrix)
    
    # Calculate the distance matrix
    distance_matrix = np.linalg.norm(p_ks_matrix[:, np.newaxis] - p_ks_matrix, axis=2)
    
    # Print the distance matrix
    print(f"Round {round_num} Distance Matrix:")
    print(distance_matrix)
    
    # Calculate and print average distances
    average_distances = np.mean(distance_matrix, axis=1)
    print(f"Round {round_num} Average distances:")
    print(average_distances)
    
    # Detect outliers using the custom DBSCAN-like function
    outliers = is_outlier(distance_matrix)
    print(f'Round {round_num}, Outliers: {outliers}')
    
    # Identify the client with the highest average distance
    max_distance_client = np.argmax(average_distances)
    print(f"Round {round_num}, Client with highest average distance: {max_distance_client}")

Round 1, Metrics: OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.8433667), ('loss', 0.5812409)]))])
Round 1 P_KS Matrix:
[[0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 6.11348569e-133 6.92665960e-165
  2.93167267e-087 7.73380966e-003 4.07559970e-091 9.16485073e-017
  1.46434551e-058 1.11892763e-154]
 [0.00000000e+000 6.11348569e-133 0.00000000e+000 2.93255464e-002
  2.22036085e-005 4.24966791e-171 1.95301357e-004 1.58617307e-056
  1.43765443e-015 2.32136078e-001]
 [0.00000000e+000 6.92665960e-165 2.93255464e-002 0.00000000e+000
  5.01874057e-013 3.86059508e-207 2.03123215e-011 6.15798172e-078
  1.73799686e-027 8.88159601e-001]
 [0.00000000e+000 2.93167267e-087 2.22036085e-005 5.01874057e-013
  0.00000000e+00

px2

In [11]:
import tensorflow_federated as tff
import tensorflow as tf
import numpy as np
from scipy.stats import ks_2samp, chi2_contingency
import nest_asyncio

nest_asyncio.apply()

# Set the local execution context
tff.backends.native.set_local_execution_context()

# Load and preprocess the MNIST dataset
def preprocess(dataset):
    return dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)))

mnist_train, mnist_test = tf.keras.datasets.mnist.load_data()
mnist_train = (mnist_train[0].reshape(-1, 28, 28), mnist_train[1])
mnist_test = (mnist_test[0].reshape(-1, 28, 28), mnist_test[1])

# Split the data into 10 clients
def create_client_data(data, labels, num_clients=10):
    client_data = []
    client_labels = []
    data_per_client = len(data) // num_clients
    for i in range(num_clients):
        client_data.append(data[i * data_per_client:(i + 1) * data_per_client])
        client_labels.append(labels[i * data_per_client:(i + 1) * data_per_client])
    return client_data, client_labels

client_data, client_labels = create_client_data(mnist_train[0], mnist_train[1])

# Define the metrics function
def perform_differential_testing(predictions_i, predictions_j):
    if predictions_i.ndim == 1:
        predictions_i = np.expand_dims(predictions_i, axis=1)
    if predictions_j.ndim == 1:
        predictions_j = np.expand_dims(predictions_j, axis=1)
    
    pred_class_i = np.argmax(predictions_i, axis=1)
    pred_class_j = np.argmax(predictions_j, axis=1)
    
    Δ_class = np.sum(pred_class_i != pred_class_j)
    Δ_score = np.sum(predictions_i != predictions_j)
    P_KS = ks_2samp(predictions_i.flatten(), predictions_j.flatten()).pvalue
    contingency = np.array([[np.sum((pred_class_i == k) & (pred_class_j == l)) for l in range(10)] for k in range(10)])
    contingency += 1  # Add-one smoothing
    P_X2 = chi2_contingency(contingency)[1]

    return Δ_class, Δ_score, P_KS, P_X2

# Create a simple model
def create_model():
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# Create a federated learning process
def model_fn():
    model = create_model()
    return tff.learning.from_keras_model(
        model,
        input_spec=(tf.TensorSpec(shape=[None, 28, 28], dtype=tf.float32),
                    tf.TensorSpec(shape=[None], dtype=tf.int32)),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Define the client optimizer function
def client_optimizer_fn():
    return tf.keras.optimizers.Nadam(learning_rate=0.001)

# Define the federated averaging process
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=client_optimizer_fn
)

# Initialize the process
state = iterative_process.initialize()

# Custom function to determine if a model is an outlier using a DBSCAN-like approach
def is_outlier(metric_data, epsilon=0.5, min_samples=2):
    num_points = metric_data.shape[0]
    distances = np.linalg.norm(metric_data[:, np.newaxis] - metric_data, axis=2)
    neighbors = np.sum(distances < epsilon, axis=1)
    outliers = neighbors < min_samples
    return outliers

# Standalone function for preprocessing
def preprocess_fn(x, y):
    return tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)

# Simulate federated training
num_rounds = 10  # Define the number of rounds
num_clients = 10  # Define the number of clients

for round_num in range(1, num_rounds + 1):
    # Create TensorFlow datasets for each client
    federated_data = [
        tf.data.Dataset.from_tensor_slices((client_data[i], client_labels[i]))
        .map(preprocess_fn)
        .batch(20)
        for i in range(num_clients)
    ]
    
    # Perform a round of federated training
    state, metrics = iterative_process.next(state, federated_data)
    print(f'Round {round_num}, Metrics: {metrics}')
    
    # Get predictions for each client
    predictions = []
    for i in range(num_clients):
        model = create_model()
        learning_rate = 10.0 if i == 0 else 0.001  # Introduce a bug in the first client
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        model.fit(client_data[i], client_labels[i], epochs=1, verbose=0)
        predictions.append(model.predict(client_data[i]))
    
    # Calculate the P_X2 matrix
    p_x2_matrix = np.zeros((num_clients, num_clients))
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            _, _, _, p_x2 = perform_differential_testing(predictions[i], predictions[j])
            p_x2_matrix[i, j] = p_x2
            p_x2_matrix[j, i] = p_x2  # Mirror the matrix
    
    # Print the P_X2 matrix
    print(f"Round {round_num} P_X2 Matrix:")
    print(p_x2_matrix)
    
    # Calculate the distance matrix
    distance_matrix = np.linalg.norm(p_x2_matrix[:, np.newaxis] - p_x2_matrix, axis=2)
    
    # Print the distance matrix
    print(f"Round {round_num} Distance Matrix:")
    print(distance_matrix)
    
    # Calculate and print average distances
    average_distances = np.mean(distance_matrix, axis=1)
    print(f"Round {round_num} Average distances:")
    print(average_distances)
    
    # Detect outliers using the custom DBSCAN-like function
    outliers = is_outlier(distance_matrix)
    print(f'Round {round_num}, Outliers: {outliers}')
    
    # Identify the client with the highest average distance
    max_distance_client = np.argmax(average_distances)
    print(f"Round {round_num}, Client with highest average distance: {max_distance_client}")

Round 1, Metrics: OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.8379167), ('loss', 0.5937404)]))])
Round 1 P_X2 Matrix:
[[0.         1.         1.         1.         1.         1.
  1.         1.         1.         1.        ]
 [1.         0.         0.89900612 0.99548838 0.69680695 0.04132197
  0.77340276 0.73489729 0.522283   0.84438677]
 [1.         0.89900612 0.         0.31105355 0.76093497 0.78284868
  0.36771522 0.66270301 0.21406162 0.43462422]
 [1.         0.99548838 0.31105355 0.         0.68218588 0.5321402
  0.72157994 0.30843208 0.37920481 0.4843528 ]
 [1.         0.69680695 0.76093497 0.68218588 0.         0.49585023
  0.061153   0.33400917 0.54655961 0.42097054]
 [1.         0.04132197 0.78284868 0.5321402  0.49585023 0.
  0.13769754 0.71986656 0.35050702 0.98268171]
 [1.         0.77340276 0.36771522 0.72157994 0.061153   0.13769754
  0.     

The metrics used in the code are calculated as follows:

Δ_class (Delta Class): This metric calculates the difference in the predicted classes between the two clients. It’s computed by comparing the argmax of the predicted probabilities for each client and counting the number of differences.

pred_class_i = np.argmax(predictions_i, axis=1)
pred_class_j = np.argmax(predictions_j, axis=1)
Δ_class = np.sum(pred_class_i != pred_class_j)
Δ_score (Delta Score): This metric calculates the sum of the absolute differences between the predicted probabilities for each client.

Δ_score = np.sum(predictions_i != predictions_j)
P_KS (Kolmogorov-Smirnov test p-value): This metric uses the Kolmogorov-Smirnov test to compare the distributions of the predicted probabilities between the two clients. The p-value of this test is used as the P_KS metric.

P_KS = ks_2samp(predictions_i.flatten(), predictions_j.flatten()).pvalue
P_X2 (Chi-square test p-value): This metric uses the Chi-square test to compare the contingency table of the predicted classes between the two clients. The p-value of this test is used as the P_X2 metric.

contingency = np.array([[np.sum((pred_class_i == k) & (pred_class_j == l)) for l in range(10)] for k in range(10)])
contingency += 1  # Add-one smoothing
P_X2 = chi2_contingency(contingency)[1]
The Δ_class and Δ_score metrics measure the differences in the predictions between the two clients, while the P_KS and P_X2 metrics measure the statistical differences in the distributions and contingency tables of the predictions, respectively.

These metrics are used to identify the client with the most divergent behavior, which is likely to be the client with the model bug. The code calculates the distance matrices based on these metrics and uses a DBSCAN-like approach to detect outliers, which are then identified as the clients with the model bug.